# 01 Import libraries

In [ ]:
import numpy as np
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [ ]:
import pandas as pd

In [ ]:
import importlib

## Import our own modules

In [ ]:
import lib
importlib.reload(lib)

<module 'lib' from '/content/Learning-the-Optimal-Solution-Path/lib/__init__.py'>

In [ ]:
from lib.lsp.basis_generator import dampen_laguerre
from lib.lsp.basis_tf_module import Basis_TF_SGD
from lib.lsp.learn_solution_path import learn_solution_path
from lib.lsp.loss_fn_lsp import reg_exp_weighted_logit
from lib.fast_tensor_data_loader import FastTensorDataLoader

# 02 Instantiate dataset

In [ ]:
# file path for Colab. May need to change this
X_df = pd.read_csv('X_processed.csv')
y_df = pd.read_csv('y_processed.csv')

In [ ]:
X = np.array(X_df)
y = np.array(y_df).squeeze()

In [ ]:
train_X = torch.tensor(X, dtype=torch.float32)
train_y = torch.tensor(y, dtype=torch.float32)

In [ ]:
# full gradient descent uses all data points
GD_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=1000, shuffle=True, )
# stochastic gradient descent uses mini-batch
SGD_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=20, shuffle=True, )
# test data
test_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=1000, shuffle=False, )

In [ ]:
lam_max = 20
lam_min = 0
input_dim = X.shape[1]
loss_fn = reg_exp_weighted_logit

In [ ]:
# Read the CSV file into a DataFrame
# truth = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/results/exact_soln_list.csv')
truth = pd.read_csv('exact_soln_list_laguerre.csv')

# Display the DataFrame
truth

,losses,theta_0,theta_1,theta_2,theta_3,theta_4,theta_5,theta_6,theta_7,theta_8,...,theta_36,theta_37,theta_38,theta_39,theta_40,theta_41,theta_42,theta_43,theta_44,theta_45
0,0.251592,-0.290164,0.186074,0.232884,-0.027476,0.166136,0.110354,0.142358,0.009975,0.157656,...,0.000471,0.023097,-0.007038,0.003151,0.004510,-0.024440,0.000471,-0.129229,-0.096597,-0.041242
1,0.251645,-0.290092,0.186073,0.232887,-0.027478,0.166136,0.110326,0.142336,0.009961,0.157669,...,0.000487,0.023069,-0.007039,0.003154,0.004514,-0.024434,0.000487,-0.129220,-0.096579,-0.041224
2,0.251698,-0.290020,0.186071,0.232891,-0.027481,0.166136,0.110298,0.142315,0.009946,0.157682,...,0.000503,0.023042,-0.007041,0.003157,0.004518,-0.024429,0.000503,-0.129210,-0.096561,-0.041206
3,0.251751,-0.289947,0.186070,0.232895,-0.027484,0.166136,0.110270,0.142294,0.009931,0.157695,...,0.000519,0.023015,-0.007042,0.003160,0.004522,-0.024423,0.000519,-0.129201,-0.096543,-0.041188
4,0.251804,-0.289874,0.186068,0.232898,-0.027487,0.166136,0.110242,0.142272,0.009916,0.157708,...,0.000534,0.022988,-0.007043,0.003162,0.004526,-0.024417,0.000534,-0.129191,-0.096525,-0.041170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,0.425709,0.608973,0.015982,0.030750,-0.012680,0.033522,-0.189898,-0.060873,-0.075246,0.138251,...,0.104496,-0.190631,-0.003899,0.016846,0.022910,0.049139,0.104496,0.117941,0.139251,0.161150
1020,0.405809,0.627298,0.006731,0.017247,-0.013635,0.025534,-0.197376,-0.075369,-0.070611,0.125166,...,0.104706,-0.193085,-0.002800,0.016029,0.021890,0.052167,0.104706,0.126507,0.144123,0.163584
1021,0.382318,0.647696,-0.004087,0.001661,-0.014793,0.016128,-0.205935,-0.094402,-0.064959,0.109066,...,0.104903,-0.195830,-0.001589,0.015089,0.020769,0.055620,0.104903,0.136154,0.149574,0.166138
1022,0.354260,0.670989,-0.016946,-0.016535,-0.016181,0.004871,-0.216011,-0.119855,-0.058109,0.089182,...,0.105111,-0.199002,-0.000252,0.014026,0.019573,0.059569,0.105111,0.147241,0.155866,0.168880


In [ ]:
selected_columns = ['theta_0', 'theta_1', 'theta_2', 'theta_3', 'theta_4',
                    'theta_5', 'theta_6', 'theta_7', 'theta_8', 'theta_9',
                    'theta_10', 'theta_11', 'theta_12', 'theta_13', 'theta_14',
                    'theta_15', 'theta_16', 'theta_17', 'theta_18', 'theta_19',
                    'theta_20', 'theta_21', 'theta_22', 'theta_23', 'theta_24',
                    'theta_25', 'theta_26', 'theta_27', 'theta_28', 'theta_29',
                    'theta_30', 'theta_31', 'theta_32', 'theta_33', 'theta_34',
                    'theta_35', 'theta_36', 'theta_37', 'theta_38', 'theta_39',
                    'theta_40', 'theta_41', 'theta_42', 'theta_43', 'theta_44',
                    'theta_45']
true_thetas = truth[selected_columns].to_numpy()
true_losses = truth['losses'].to_numpy()

# 03 SGD with Diminishing LR by Distance Diagnostic

Recall that our method runs SGD over random $\tilde λ$'s with a linear basis $\Phi(\tilde \lambda)$ of our choice. We want to approximate $\theta$ with $\Phi(\lambda)\beta$, so the objective function is $\min_\beta h(\Phi(\tilde\lambda)\beta, \tilde\lambda) = (1-\tilde\lambda) BCE(X_\text{pass}\Phi(\tilde\lambda)\beta,\ y_\text{pass}) + \tilde\lambda BCE(X_\text{fail}\Phi(\tilde\lambda)\beta,\ y_\text{fail})$. For each batch of training data set, we randomize $\tilde\lambda$.

We use diminishing learning rate for better demonstrate convergence. If we use a constant learning rate, the solution path error will eventually do a random walk after descending to a certain threshold value.

Set weighted_avg to 'False' see this random walk.

In [ ]:
max_epochs = 6000

In [ ]:
phi_lam = dampen_laguerre

In [ ]:
def thresh_basis(basis_dim):
    return -1

## num basis func = 5

In [ ]:
basis_dim = 5
init_lr = 0.0625

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr, itr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min, lam_max, true_losses, init_lr=init_lr,
                                                               diminish=True, q=1.5, thresh_basis=thresh_basis,
                                                               record_frequency=10, distribution='exponential',
                                                               device=device, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# epoch: 100	 sup error: 0.06728923320770264
--------approximate solution path for # itr = 200 complete--------
# epoch: 200	 sup error: 0.061814248561859186
--------approximate solution path for # itr = 300 complete--------
# epoch: 300	 sup error: 0.06067377328872686
--------approximate solution path for # itr = 400 complete--------
# epoch: 400	 sup error: 0.10549074411392217
--------approximate solution path for # itr = 500 complete--------
# epoch: 500	 sup error: 0.08142107725143438
--------approximate solution path for # itr = 600 complete--------
# epoch: 600	 sup error: 0.09814649820327764
--------approximate solution path for # itr = 700 complete--------
# epoch: 700	 sup error: 0.10419979691505438
--------approximate solution path for # itr = 800 complete--------
# epoch: 800	 sup error: 0.10402005910873419
--------approximate solution path for # itr = 900 complete--------
# epoch: 900	 sup error: 0.095866650

In [ ]:
file_path = 'LSP_results_exact_fixed_basis.csv'

LSP_results_exact_fixed_basis = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err_5'])

# Save the DataFrame to a CSV file
LSP_results_exact_fixed_basis.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,num_itr,sup_err_5
0,10.0,0.170329
1,20.0,0.126928
2,30.0,0.172975
3,40.0,0.283612
4,50.0,0.284675
...,...,...
595,5960.0,0.093463
596,5970.0,0.093419
597,5980.0,0.093537
598,5990.0,0.093370


## num basis func = 7

In [ ]:
basis_dim = 7
init_lr = 0.0625

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr, itr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min, lam_max, true_losses, init_lr=init_lr,
                                                               diminish=True, q=1.5, thresh_basis=thresh_basis,
                                                               record_frequency=10, distribution='exponential',
                                                               device=device, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# epoch: 100	 sup error: 0.08830827474594116
--------approximate solution path for # itr = 200 complete--------
# epoch: 200	 sup error: 0.051316797733306885
--------approximate solution path for # itr = 300 complete--------
# epoch: 300	 sup error: 0.03262564539909368
--------approximate solution path for # itr = 400 complete--------
# epoch: 400	 sup error: 0.03091061115264898
--------approximate solution path for # itr = 500 complete--------
# epoch: 500	 sup error: 0.027068316936492975
--------approximate solution path for # itr = 600 complete--------
# epoch: 600	 sup error: 0.026059836149215754
--------approximate solution path for # itr = 700 complete--------
# epoch: 700	 sup error: 0.024442106485366877
--------approximate solution path for # itr = 800 complete--------
# epoch: 800	 sup error: 0.01707234978675848
--------approximate solution path for # itr = 900 complete--------
# epoch: 900	 sup error: 0.013546

In [ ]:
file_path = 'LSP_results_exact_fixed_basis.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df['sup_err_7'] = sup_err_history

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

## num basis func = 9

In [ ]:
basis_dim = 9
init_lr = 0.0625

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr, itr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min, lam_max, true_losses, init_lr=init_lr,
                                                               diminish=True, q=1.5, thresh_basis=thresh_basis,
                                                               record_frequency=10, distribution='exponential',
                                                               device=device, trace_frequency=100)
sup_err_history = np.array(sup_err_history)

--------approximate solution path for # itr = 100 complete--------
# epoch: 100	 sup error: 0.042099952697753906
--------approximate solution path for # itr = 200 complete--------
# epoch: 200	 sup error: 0.02271133661270147
--------approximate solution path for # itr = 300 complete--------
# epoch: 300	 sup error: 0.04453656077384954
--------approximate solution path for # itr = 400 complete--------
# epoch: 400	 sup error: 0.02499511837959295
--------approximate solution path for # itr = 500 complete--------
# epoch: 500	 sup error: 0.015968173742294367
--------approximate solution path for # itr = 600 complete--------
# epoch: 600	 sup error: 0.011430442333221491
--------approximate solution path for # itr = 700 complete--------
# epoch: 700	 sup error: 0.009686499834060724
--------approximate solution path for # itr = 800 complete--------
# epoch: 800	 sup error: 0.012021511793136652
--------approximate solution path for # itr = 900 complete--------
# epoch: 900	 sup error: 0.00985

In [ ]:
file_path = 'LSP_results_exact_fixed_basis.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df['sup_err_9'] = sup_err_history

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)